In [1]:
import os
import keras.backend as K

In [50]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten
from keras.layers import Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from tensorflow.keras.optimizers import RMSprop, SGD, Adam

In [34]:
channel_axis = -1
K.set_image_data_format('channels_last')
channel_first = False
def wbconv_init(a):
    k = RandomNormal(0, 0.02)(a)
    k.conv_weight = True
    return k

conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1, 0.02)

In [35]:
def conv2d(f, *a, **k):
    return Conv2D(f, kernel_initializer = conv_init, *a, **k)

def batchnorm():
    return BatchNormalization(momentum=0.9, axis=channel_axis, epsilon=1.01e-5,
                             gamma_initializer=gamma_init)

In [36]:
def basic_discriminator(nc_in, ndf, max_layers=3, use_sigmoid=True):
    i = Input(shape=(None, None, nc_in))
    h = conv2d(ndf, kernel_size=4, strides=2, padding='same', name='1')(i)
    h = LeakyReLU(alpha=0.2)(h)
    
    for layer in range(1, max_layers):
        out_feat = ndf * min(2 ** layer, 8)
        h = conv2d(out_feat, kernel_size=4,strides=2, padding='same', use_bias=False,
                  name='pyramid.{0}'.format(layer))(h)
        h = batchnorm()(h, training=1)
        h = LeakyReLU(alpha=0.2)(h)
        
    out_feat = ndf * min(2 ** max_layers, 8)
    h = ZeroPadding2D(1)(h)
    h = conv2d(out_feat, kernel_size=4, use_bias=False, name='pyramid_last')(h)
    h = batchnorm()(h, training=1)
    h = LeakyReLU(alpha=0.2)(h)
    
    h = ZeroPadding2D(1)(h)
    h = conv2d(1, kernel_size=4, name='final'.format(out_feat, 1),
              activation='sigmoid' if use_sigmoid else None)(h)
    return Model(inputs=[i], outputs=h)

In [46]:
def unet_g(isize, nc_in=3, nc_out=3, ngf=64, fixed_input_size=True):
    max_nf = 8 * ngf
    def block(x, s, nf_in, use_batchnorm=True, nf_out=None, nf_next=None):
        assert s >= 2 and s%2 == 0
        if nf_next is None:
            nf_next = min(nf_in * 2, max_nf)
        if nf_out is None:
            nf_out = nf_in
        h = conv2d(nf_next, kernel_size=4, strides=2, use_bias=(not(use_batchnorm and s >2)),
                   padding='same', name='conv_{0}'.format(s))(x)
        if s > 2:
            if use_batchnorm:
                h = batchnorm()(h, training=1)
            h2 = LeakyReLU(alpha=0.2)(h)
            h2 = block(h2, s//2, nf_next)
            h = Concatenate(axis=channel_axis)([h, h2])
        h = Activation('relu')(h)
        h = Conv2DTranspose(nf_out, kernel_size=4, strides=2, use_bias=not use_batchnorm, kernel_initializer=conv_init,
                           name='convt.{0}'.format(s))(h)
        h = Cropping2D(1)(h)
        if use_batchnorm:
            h = batchnorm()(h, training=1)
        if s <= 8:
            h = Dropout(0.5)(h, training=1)
        return h
    s = isize if fixed_input_size else None
    if channel_first:
        _ = inputs = Input(shape=(nc_in, s, s))
    else:
        _ = inputs = Input(shape=(s, s, nc_in))
    _ = block(_, isize, nc_in, False, nf_out=nc_out, nf_next=ngf)
    _ = Activation('tanh')(_)
    return Model(inputs=inputs, outputs=[_])

In [47]:
nc_in = 3
nc_out = 3
ngf = 64
ndf = 64
use_lsgan = True
lam = 10 if use_lsgan else 100
loaSize = 143
imageSize = 128
bachSize = 1
lrD = 2e-4
lrG = 2e-4

In [48]:
netDA = basic_discriminator(nc_in, ndf, use_sigmoid = not use_lsgan)
netDB = basic_discriminator(nc_out, ndf, use_sigmoid = not use_lsgan)
netDA.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, None, None, 3)]   0         
_________________________________________________________________
1 (Conv2D)                   (None, None, None, 64)    3136      
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, None, None, 64)    0         
_________________________________________________________________
pyramid.1 (Conv2D)           (None, None, None, 128)   131072    
_________________________________________________________________
batch_normalization_15 (Batc (None, None, None, 128)   512       
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, None, None, 128)   0         
_________________________________________________________________
pyramid.2 (Conv2D)           (None, None, None, 256)  

In [49]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


netGB = unet_g(imageSize, nc_in, nc_out, ngf)
netGA = unet_g(imageSize, nc_out, nc_in, ngf)
#SVG(model_to_dot(netG, show_shapes=True).create(prog='dot', format='svg'))
netGA.summary()

Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv_128 (Conv2D)               (None, 64, 64, 64)   3136        input_16[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_36 (LeakyReLU)      (None, 64, 64, 64)   0           conv_128[0][0]                   
__________________________________________________________________________________________________
conv_64 (Conv2D)                (None, 32, 32, 128)  131072      leaky_re_lu_36[0][0]             
______________________________________________________________________________________

In [54]:
if use_lsgan:
    loss_fn = lambda output, target: K.mean(K.abs(K.square(output-target)))
else:
    loss_fn = lambda output, target: -K.mean(K.log(output+1e-12)* target + K.log(1-output+1e-12)*(1-target))

def cycle_variables(netG1, netG2):
    real_input = netG1.inputs[0]
    fake_output = netG1.outputs[0]
    rec_input = netG2([fake_output])
    fn_generate = K.function([real_input], [fake_output, rec_input])
    return real_input, fake_output, rec_input, fn_generate

real_A, fake_B, rec_A, cycleA_generate = cycle_variables(netGB, netGA)
real_B, fake_A, rec_B, cycleB_generate = cycle_variables(netGA, netGB)

In [55]:
def D_loss(netD, real, fake, rec):
    output_real = netD([real])
    output_fake = netD([fake])
    loss_D_real = loss_fn(output_real, K.ones_like(output_real))
    loss_D_fake = loss_fn(output_fake, K.zeros_like(output_fake))
    loss_G = loss_fn(output_fake, K.ones_like(output_fake))
    loss_D = loss_D_real + loss_D_fake
    loss_cyc = K.mean(K.abs(rec-real))
    return loss_D, loss_G, loss_cyc

In [56]:
loss_DA, loss_GA, loss_cycA = D_loss(netDA, real_A, fake_A, rec_A)
loss_DB, loss_GB, loss_cycB = D_loss(netDB, real_B, fake_B, rec_B)
loss_cyc = loss_cycA+loss_cycB

In [63]:
loss_G = loss_GA+loss_GB+lam*loss_cyc
loss_D = loss_DA+loss_DB

weightsD = netDA.trainable_weights + netDB.trainable_weights
weightsG = netGA.trainable_weights + netGB.trainable_weights

training_updates = Adam(lr=lrD, beta_1=0.5).get_updates(loss_D, weightsD)
netD_train = K.function([real_A, real_B],[loss_DA/2, loss_DB/2], training_updates)
training_updates = Adam(lr=lrG, beta_1=0.5).get_updates(weightsG,[], loss_G)
netG_train = K.function([real_A, real_B], [loss_GA, loss_GB, loss_cyc], training_updates)

ValueError: `updates` argument is not supported during eager execution. You passed: [<tf.Operation 'AssignAddVariableOp_1' type=AssignAddVariableOp>]

In [ ]:
from PIL import Image
import numpy as np
import glob
from random import randint, shuffle